# 🔬 Analyse Exploratoire des Labels Générés

**Objectif :** Valider la cohérence, la distribution et la pertinence des labels générés par le pipeline de données (`utils/labeling.py`) avant de les utiliser pour l'entraînement du modèle.

**Dataset :** `data/processed/final_dataset.parquet` (supposé généré par `data/pipelines/data_pipeline.py`)

**Labels à analyser :**
- `label_binary` (Généré par `generate_binary_labels`)
- `label_multi_class` (Généré par `generate_multiclass_labels`, correspond à `signal_trading`)
- `label_market_regime` (Généré par `generate_market_regimes`)
- `potential_tp_long`, `potential_sl_long`, `potential_tp_short`, `potential_sl_short` (Générés par `calculate_tp_sl_targets`, correspondent à `level_tp`/`level_sl`)
- `horizon_signal` (Note : Cette colonne n'est pas explicitement générée par `utils/labeling.py` dans sa version actuelle. Si elle est ajoutée par une autre partie du pipeline, elle sera analysée ici. Sinon, cette section sera omise.)

## 1. Configuration et Chargement des Données

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configuration de l'affichage
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
warnings.filterwarnings('ignore', category=FutureWarning) # Ignore future warnings from seaborn/pandas

# Chemin vers le dataset final (ajuster si nécessaire)
DATASET_PATH = '../data/processed/final_dataset.parquet'

# Charger les données
try:
    df = pd.read_parquet(DATASET_PATH)
    print(f"Dataset chargé depuis {DATASET_PATH}")
    print(f"Shape: {df.shape}")
    print("Colonnes disponibles:", df.columns.tolist())
    # Assurer que l'index est un DatetimeIndex si ce n'est pas déjà le cas
    if not isinstance(df.index, pd.DatetimeIndex):
         # Tenter de convertir la colonne 'timestamp' ou l'index
        if 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df = df.set_index('timestamp')
            print("Index 'timestamp' converti en DatetimeIndex.")
        else:
             try:
                 df.index = pd.to_datetime(df.index)
                 print("Index existant converti en DatetimeIndex.")
             except Exception as e:
                 print(f"Avertissement: Impossible de convertir l'index en DatetimeIndex: {e}")

    # Afficher les premières lignes et informations
    display(df.head())
    display(df.info())
    display(df.describe(include='all'))

except FileNotFoundError:
    print(f"ERREUR: Le fichier {DATASET_PATH} n'a pas été trouvé.")
    print("Veuillez exécuter le pipeline de données (`data/pipelines/data_pipeline.py`) pour le générer.")
    df = None # Définir df à None pour éviter les erreurs dans les cellules suivantes
except Exception as e:
    print(f"Une erreur est survenue lors du chargement des données: {e}")
    df = None

## 2. Analyse du Label : `label_multi_class` (Signal de Trading)

- **Définition :** Signal de trading à 5 classes basé sur le changement de prix futur.
- **Classes :** -2 (Strong Sell), -1 (Sell), 0 (Hold), 1 (Buy), 2 (Strong Buy).
- **Méthode :** `generate_multiclass_labels` utilisant des seuils sur le rendement à un horizon donné.

In [ ]:
if df is not None and 'label_multi_class' in df.columns:
    label_col = 'label_multi_class'
    print(f"\n--- Analyse de '{label_col}' ---")

    # Distribution des classes
    plt.figure(figsize=(10, 5))
    sns.countplot(x=label_col, data=df, palette='viridis')
    plt.title(f'Distribution des classes pour {label_col}')
    plt.xlabel('Classe de Signal')
    plt.ylabel('Nombre d\'occurrences')
    plt.show()

    # Afficher les pourcentages
    class_counts = df[label_col].value_counts(normalize=True).sort_index()
    print("Distribution en pourcentage:")
    print(class_counts * 100)

    # Vérifier le déséquilibre
    if class_counts.min() < 0.05: # Seuil arbitraire de 5%
        print("\nAvertissement: Déséquilibre potentiel des classes détecté.")

    # Relation avec le prix (visualisation sur un segment)
    # Note: La relation directe est complexe car le label dépend du futur
    if 'close' in df.columns and isinstance(df.index, pd.DatetimeIndex):
        sample_df = df.last('30D') # Visualiser les 30 derniers jours
        if not sample_df.empty:
            fig, ax1 = plt.subplots(figsize=(15, 7))

            ax1.plot(sample_df.index, sample_df['close'], label='Prix (Close)', color='blue', alpha=0.8)
            ax1.set_xlabel('Date')
            ax1.set_ylabel('Prix Close', color='blue')
            ax1.tick_params(axis='y', labelcolor='blue')

            ax2 = ax1.twinx() # Partager le même axe x
            # Utiliser des marqueurs pour les signaux non-Hold
            signals = sample_df[sample_df[label_col] != 0]
            buy_signals = signals[signals[label_col] > 0]
            sell_signals = signals[signals[label_col] < 0]

            ax2.scatter(buy_signals.index, buy_signals[label_col], label='Buy/Strong Buy Signal', color='green', marker='^', s=100, alpha=0.7)
            ax2.scatter(sell_signals.index, sell_signals[label_col], label='Sell/Strong Sell Signal', color='red', marker='v', s=100, alpha=0.7)
            ax2.set_ylabel('Signal Class', color='black')
            ax2.tick_params(axis='y', labelcolor='black')
            ax2.set_ylim(-2.5, 2.5) # Ajuster les limites pour la visibilité
            ax2.grid(False)

            plt.title(f'Prix Close et Signaux {label_col} (30 derniers jours)')
            fig.tight_layout()
            # Combiner les légendes
            lines, labels = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax2.legend(lines + lines2, labels + labels2, loc='upper left')
            plt.show()
        else:
            print("Pas assez de données dans les 30 derniers jours pour visualiser la relation avec le prix.")
else:
    print(f"Colonne 'label_multi_class' non trouvée ou DataFrame non chargé.")

## 3. Analyse du Label : `label_market_regime`

- **Définition :** Catégorisation de l'état actuel du marché.
- **Classes :** Numéros de cluster (ex: 0, 1, 2) représentant différents régimes (tendance, range, volatile, etc.).
- **Méthode :** `generate_market_regimes` utilisant K-Means sur des features sélectionnées (ex: volatilité, volume).

In [ ]:
if df is not None and 'label_market_regime' in df.columns:
    label_col = 'label_market_regime'
    print(f"\n--- Analyse de '{label_col}' ---")

    # Distribution des régimes
    plt.figure(figsize=(10, 5))
    # S'assurer que la colonne est traitée comme catégorielle pour countplot
    regime_data = df[label_col].dropna().astype(int)
    if not regime_data.empty:
        sns.countplot(x=regime_data, palette='Set2')
        plt.title(f'Distribution des Régimes de Marché ({label_col})')
        plt.xlabel('Régime (Cluster)')
        plt.ylabel('Nombre d\'occurrences')
        plt.show()

        # Afficher les pourcentages
        regime_counts = regime_data.value_counts(normalize=True).sort_index()
        print("Distribution en pourcentage:")
        print(regime_counts * 100)
    else:
        print("Aucune donnée de régime de marché valide trouvée.")

    # Visualisation des régimes superposés au prix (sur un segment)
    if 'close' in df.columns and isinstance(df.index, pd.DatetimeIndex):
        sample_df = df.last('90D') # Visualiser les 90 derniers jours
        if not sample_df.empty and not sample_df[label_col].dropna().empty:
            plt.figure(figsize=(15, 7))
            sns.lineplot(data=sample_df, x=sample_df.index, y='close', label='Prix Close', color='black', alpha=0.5, zorder=1)

            # Utiliser scatter pour colorer par régime
            # S'assurer que les régimes sont numériques pour la palette
            valid_regimes = sample_df.dropna(subset=[label_col])
            valid_regimes[label_col] = valid_regimes[label_col].astype(int)

            scatter = plt.scatter(valid_regimes.index, valid_regimes['close'], c=valid_regimes[label_col],
                                cmap='viridis', label='Régime de Marché', s=10, zorder=2)

            plt.title('Prix Close et Régimes de Marché (90 derniers jours)')
            plt.xlabel('Date')
            plt.ylabel('Prix Close')
            # Créer une légende pour les régimes
            handles, _ = scatter.legend_elements(prop='colors', alpha=0.8)
            regime_labels = [f'Régime {i}' for i in sorted(valid_regimes[label_col].unique())]
            plt.legend(handles, regime_labels, title="Régimes")
            plt.show()
        else:
            print("Pas assez de données ou de régimes valides dans les 90 derniers jours pour visualiser.")
else:
    print(f"Colonne 'label_market_regime' non trouvée ou DataFrame non chargé.")

## 4. Analyse des Labels : Niveaux SL/TP Potentiels

- **Définition :** Niveaux de prix potentiels pour Stop Loss (SL) et Take Profit (TP).
- **Colonnes :** `potential_tp_long`, `potential_sl_long`, `potential_tp_short`, `potential_sl_short`.
- **Méthode :** `calculate_tp_sl_targets` utilisant l'ATR et des multiplicateurs.

In [ ]:
sl_tp_cols = ['potential_tp_long', 'potential_sl_long', 'potential_tp_short', 'potential_sl_short']

if df is not None and all(col in df.columns for col in sl_tp_cols) and 'close' in df.columns:
    print(f"\n--- Analyse des Niveaux SL/TP ---")

    # Vérifier les valeurs manquantes
    print("Pourcentage de valeurs manquantes pour SL/TP:")
    print(df[sl_tp_cols].isnull().mean() * 100)

    # Visualiser les niveaux par rapport au prix (sur un segment)
    if isinstance(df.index, pd.DatetimeIndex):
        sample_df = df.last('30D') # Visualiser les 30 derniers jours
        if not sample_df.empty:
            plt.figure(figsize=(15, 7))
            plt.plot(sample_df.index, sample_df['close'], label='Prix Close', color='black', linewidth=2, zorder=5)

            # Niveaux Long
            plt.plot(sample_df.index, sample_df['potential_tp_long'], label='TP Long Potentiel', color='green', linestyle='--', alpha=0.7)
            plt.plot(sample_df.index, sample_df['potential_sl_long'], label='SL Long Potentiel', color='red', linestyle='--', alpha=0.7)

            # Niveaux Short (optionnel, peut surcharger le graphique)
            # plt.plot(sample_df.index, sample_df['potential_tp_short'], label='TP Short Potentiel', color='lime', linestyle=':', alpha=0.6)
            # plt.plot(sample_df.index, sample_df['potential_sl_short'], label='SL Short Potentiel', color='darkred', linestyle=':', alpha=0.6)

            plt.title('Prix Close et Niveaux SL/TP Potentiels (Long) (30 derniers jours)')
            plt.xlabel('Date')
            plt.ylabel('Prix')
            plt.legend()
            plt.show()

            # Analyser la distance SL/TP par rapport au prix
            tp_dist_long = sample_df['potential_tp_long'] - sample_df['close']
            sl_dist_long = sample_df['close'] - sample_df['potential_sl_long']

            plt.figure(figsize=(12, 5))
            sns.histplot(tp_dist_long, color='green', label='Distance TP Long', kde=True, alpha=0.6)
            sns.histplot(sl_dist_long, color='red', label='Distance SL Long', kde=True, alpha=0.6)
            plt.title('Distribution des Distances SL/TP (Long)')
            plt.xlabel('Distance par rapport au prix Close')
            plt.legend()
            plt.show()

            print("Statistiques descriptives des distances SL/TP (Long):")
            display(pd.DataFrame({'TP_Distance': tp_dist_long, 'SL_Distance': sl_dist_long}).describe())

        else:
            print("Pas assez de données dans les 30 derniers jours pour visualiser les niveaux SL/TP.")
else:
    print(f"Colonnes SL/TP ou 'close' non trouvées ou DataFrame non chargé.")

## 5. Analyse du Label : `label_binary`

- **Définition :** Signal de trading binaire simple.
- **Classes :** 1 (Buy), -1 (Sell), 0 (Hold).
- **Méthode :** `generate_binary_labels` basé sur un seuil de rendement futur.

In [ ]:
if df is not None and 'label_binary' in df.columns:
    label_col = 'label_binary'
    print(f"\n--- Analyse de '{label_col}' ---")

    # Distribution des classes
    plt.figure(figsize=(8, 4))
    sns.countplot(x=label_col, data=df, palette='coolwarm')
    plt.title(f'Distribution des classes pour {label_col}')
    plt.xlabel('Classe de Signal Binaire')
    plt.ylabel('Nombre d\'occurrences')
    plt.show()

    # Afficher les pourcentages
    class_counts_bin = df[label_col].value_counts(normalize=True).sort_index()
    print("Distribution en pourcentage:")
    print(class_counts_bin * 100)

    # Vérifier le déséquilibre
    if class_counts_bin.min() < 0.05: # Seuil arbitraire de 5%
        print("\nAvertissement: Déséquilibre potentiel des classes détecté.")
else:
    print(f"Colonne 'label_binary' non trouvée ou DataFrame non chargé.")

## 6. Analyse du Label : `horizon_signal` (Optionnel)

- **Définition :** Horizon temporel associé au signal de trading (si applicable).
- **Méthode :** (Non défini dans `utils/labeling.py` actuellement).

In [ ]:
if df is not None and 'horizon_signal' in df.columns:
    label_col = 'horizon_signal'
    print(f"\n--- Analyse de '{label_col}' ---")

    # Distribution des horizons (si catégoriel ou discret)
    if df[label_col].nunique() < 30: # Si peu de valeurs uniques
        plt.figure(figsize=(10, 5))
        sns.countplot(x=label_col, data=df, palette='crest')
        plt.title(f'Distribution des horizons de signal ({label_col})')
        plt.xlabel('Horizon')
        plt.ylabel('Nombre d\'occurrences')
        plt.show()
    else: # Si continu ou beaucoup de valeurs
        plt.figure(figsize=(10, 5))
        sns.histplot(df[label_col].dropna(), kde=True, bins=30)
        plt.title(f'Distribution des horizons de signal ({label_col})')
        plt.xlabel('Horizon')
        plt.ylabel('Fréquence')
        plt.show()

    print("Statistiques descriptives:")
    display(df[label_col].describe())
else:
    print(f"Colonne 'horizon_signal' non trouvée ou DataFrame non chargé. Analyse omise.")

## 7. Conclusion de l'Analyse Exploratoire

- Résumer ici les observations clés sur la distribution, la cohérence et les éventuels problèmes (déséquilibre, valeurs aberrantes, etc.) des labels.

In [ ]:
# Espace pour des analyses supplémentaires ou des conclusions finales